# Cholesky method

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi

## Historical background
André-Louis Cholesky (1875-1918), student at the école polytechnique (X1895), originally designed his method for solving linear problems for a problem emerging in the field of topography. He worked on triangluations to draw maps of different countries. 

A triangulation is of a set of triangles recovering the domain to be mapped. To each node of this triangulation is associated its position and its altitude, which are required to draw the map. On the field, only distances and angles can be evaluated, the positions and the altitudes need to be computed based on these observations. One verifies that, using only the length of one side of a triangle and two of its angles, one may reconstruct all the others. To simplify, assuming that the lengths and angles are linear functions of positions and altitude, one rewrites this problem under the form \begin{equation}A V = b\end{equation} where $b$ is a vector of the lengths and angles observed, $A$ is a matrix, and $V$ is the vector of the exact positions and altitudes of the nodes that we seek.  
 
However, in practice, the evaluations of lengths and angles can be inexact due to human or instrument inaccuracy. Thus, more evaluations are performed, e.g. evaluating all the angles and the lengths of all sides of a triangle. In the new problem $AV=b$, we have more data $b\in\mathbb{R}^M$ than unknowns $V\in\mathbb{R}^N$, i.e. $M>N$ and the matrix $A\in\mathbb{R}^{M \times N}$ is rectangular. This problem does not have a solution in general (only if all observations agrees exactly with each others). Instead, we minimize the error $\epsilon(V) = \|A V - b\|_2^2$, which is known (admitted here) to have a solution. This solution satisfies the so-called normal equation 
\begin{equation} A^T A V = A^T b. \end{equation}

We discuss here numerical solution of this new linear equation.

## Requirements

1) Prove that $B = A^T A$ is square, symmetric and positive. Give a condition on the columns $C^{j}$ (s.t. $C_i^j = A_{i,j}$) for $B$ to be definite (which will be assumed in the next questions).

Answer:

 

2) Here, we consider that $A$ is square and $A^T A$ is definite. What can you say about $A$? Prove that the problem $A V = b$ and the problem $(A^T A) V = A^T b$ have the same solution $V$.

Answer:

## First decomposition

3) Cholesky remarked that if $A^T$ was square and lower triangular (call it $L$), then solving $A^T A V = A^T V$ could be performed in two parts very easily (and accessible with the technology of its time). Then, he suggested to decompose any symmetric positive definite $B$ (in the present case $A^T A$) as the product of a lower triangular matrix and its transposed $B = L L^T$. 

a)Write down the coefficients $(L L^T)_{i,j}$ as a function of the coefficient $L_{i,j}$. 

b)Write the diagonal coefficient $L_{i,i}$ as a functions of $B_{i,i}$ and of $L_{i,j}$ with $j<i$ (hints: represent on paper the matrix $L$). 

c)Write the non-diagonal coefficient $L_{j,i}$ with $j<i$ as a function of $B$ and of the relevant coefficients of $L$. 

Answer:

a)

b)

c)

4) Doing the computations in the appropriate order, write an algorithm for the computation of all the entries of $L$. 
Explain your choice for doing the computations in this order.

Answer: 





5) a) Implement this algorithm to compute a lower triangular matrix $L \in\mathbb{R}^{N \times N}$ such that $L L^T = B$ for a symmetric positive definite matrix $B$. 

b) Test it with the matrix $B\in\mathbb{R}^{N\times N}$ defined such that \begin{equation}B_{i,j} = 1 + 2N \delta_{i,j}\end{equation} for $N=10$. Check that your $L$ matrix does satisfy $B=LL^T$.

In [3]:
def LL_decomp(B):
    ### implement your algorithm to compute this LL^T decomposition 
    N = len(B[1,:]) 
    L = np.eye(N) 
    for i in range(N): 
        for j in range(i): 
            L[i,j] = (B[i,j] - np.sum([L[i,k] * L[j,k] for k in range(j)])) / L[j,j] 
        L[i,i] = np.sqrt(B[i,i] - np.sum([L[i,k] * L[i,k] for k in range(i)])) 
    
    return L

## Second decomposition

6) Now we aim to adapt this algorithm to symmetric matrices that may not be positive definite. For this purpose, we seek a $L D L^T$ decomposition where $L$ is still lower triangular, it has only ones on the diagonal and $D$ is a diagonal matrix (not necessarily positive). 

Write an alorithm computing $L$ and $D$ for a symmetric matrix $B = L D L^T$. Explain the choice of the order in which you do the computations

Answer:

7) Implement and test this algorithm with the matrix of the question 4). Check that the matrices obtained with your algorithm do satisfy $B = LDL^T$. 

In [4]:
def LDL_decomp(B):
    ### implement your algorithm to obtain this LDL^T decomposition 
    N = len(B[1,:]) 
    L = np.eye(N) 
    D = np.ones(N) 
    for j in range(N): 
        D[j,j] = B[j,j] - np.sum([D[k,k] * L[j,k]**2 for k in range(j)]) 
        for i in range(j+1, N): 
            L[i,j] = (B[i,j] - np.sum([D[k,k] * L[i,k] * L[j,k] for k in range(j)])) / D[j,j] 
    
    return L, D

8) If $B = L^1 (L^1)^T = L^2 D (L^2)^T$ is positive definite, how can you relate $L^1$ from the first decomposition to $L^2$ and $D$ from the second one (express one as a function of the others)?

Answer:

9) Compare the number of operations required to compute $L^1 (L^1)^T$ and $L^2 D (L^2)^T$ decompositions (square root is one operation). 
Compare the number of entries of $L^1$ and of $D$ and $L^2$ that you need to construct these matrices.

Answer:

## Application to least square problem and curve fitting

As in the introduction, suppose now that we have more data than unknowns. We aim to construct a curve of the form \begin{equation} f(x) = \exp\left(\sum\limits_{i=0}^{N-1} V_i x^i\right) \end{equation} passing as close as possible to the data $(b_j)_{j=1,\dots,M}$ at the positions $(x_j)_{j=1,\dots,M}$. 
In practice, we seek the coefficients $(V_i)_{i=0,\dots,N-1}$. 

- Write an equation equivalent to $\exp\left(\sum\limits_{i=0}^{N-1} V_i x_j^i\right) = b_j$ in which the left hand-side is a linear function of the $V_i$.

- Then, write this problem as linear problem $AV = c$ with a non-square matrix $A$ and write down the entries of $A$ and $c$. 

- Apply the previous decompositions (Cholesky and $L D L^T$) to the matrix $A^T A$.

- Implement back and forward substitution algorithm to solve the normal equation $A^T A V = A^T c$.

- Test it with the given vector of data for $N=10$. Plot on the same graph the points $(x_j,b_j)$ and your solution $f(x)$.

Answer:

- 

- 

In [ ]:
N_data   = 100
N_interp = 10

x = 0.5 * (1 + np.cos(pi*(2*np.array(range(N_data))-1)/N_data))
b = np.array([np.exp(np.cos(y) - np.cos(5.*y/(2-y)**2)) for y in x])

# A =
# B = A^T A
# L = LL_decomp(B)


def f(x):
    # return the approximated function f(x) evaluated at all positions x
    return x

plt.figure(1)
plt.scatter(x, b, color='red',  label="Data", marker='.', s=0.5)
plt.plot(x, f(x), color='blue', label='Interpolation')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.show()